<a href="https://colab.research.google.com/github/jivitheshreddy/INFO-5731-Srping2023/blob/main/In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [12]:
# Write your code here
import pandas as pd
df=pd.read_csv('amazon_reviews.csv')

In [17]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,reviewTextprocessed
0,0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09,we got this gps for my husband who is an (otr)...
1,1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09,i'm a professional otr truck driver and i boug...
2,2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09,well what can i say i've had this unit in my ...
3,3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1.290557e+09,not going to write a long review even thought ...
4,4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1.317254e+09,i've had mine for a year and here's what we go...


In [16]:
# Load the regular expression library
import re
# Remove punctuation
df['reviewTextprocessed'] = \
df['reviewTextprocessed'] = df['reviewText'].map(lambda x: re.sub('[,\.!?]', '', x) if isinstance(x, str) else '')
# Convert the titles to lowercase
df['reviewTextprocessed'] = \
df['reviewTextprocessed'].map(lambda x: x.lower())
# Print out the first rows of papers
df['reviewTextprocessed'].head()

0    we got this gps for my husband who is an (otr)...
1    i'm a professional otr truck driver and i boug...
2    well what can i say  i've had this unit in my ...
3    not going to write a long review even thought ...
4    i've had mine for a year and here's what we go...
Name: reviewTextprocessed, dtype: object

In [18]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.reviewTextprocessed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['got', 'gps', 'husband', 'otr', 'road', 'trucker', 'impressed', 'shipping', 'time', 'arrived', 'days', 'earlier', 'expected', 'within', 'week', 'however', 'started', 'freezing', 'could', 'glitch', 'unit', 'worked', 'great', 'worked', 'work', 'great', 'normal', 'person', 'well', 'trucker']


In [26]:
import gensim.corpora as corpora
# Create Dictionary
dictionary = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1)]


In [28]:
# build LDA model and determine optimal number of topics
coherence_scores = {}
for k in range(2, 10):
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=k)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=dictionary, coherence='c_v')
    coherence_scores[k] = coherence_model_lda.get_coherence()
optimal_k = max(coherence_scores, key=coherence_scores.get)

In [29]:
# summarize topics
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=optimal_k)
topics = lda_model.show_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.014*"tv" + 0.011*"mount" + 0.007*"one" + 0.007*"great" + 0.007*"wall" + 0.005*"nook" + 0.005*"get" + 0.005*"would" + 0.005*"works" + 0.005*"well"')
(1, '0.015*"nook" + 0.009*"books" + 0.008*"one" + 0.006*"like" + 0.006*"great" + 0.006*"book" + 0.005*"buy" + 0.004*"also" + 0.004*"good" + 0.004*"reading"')
(2, '0.016*"nook" + 0.010*"tv" + 0.010*"books" + 0.008*"like" + 0.007*"great" + 0.007*"screen" + 0.006*"mount" + 0.006*"would" + 0.006*"kindle" + 0.005*"price"')
(3, '0.025*"nook" + 0.009*"kindle" + 0.009*"tablet" + 0.008*"one" + 0.008*"books" + 0.007*"get" + 0.007*"screen" + 0.006*"like" + 0.006*"read" + 0.006*"great"')
(4, '0.015*"nook" + 0.007*"one" + 0.007*"tablet" + 0.006*"screen" + 0.006*"like" + 0.005*"device" + 0.005*"tv" + 0.005*"kindle" + 0.005*"would" + 0.005*"card"')
(5, '0.011*"nook" + 0.008*"books" + 0.008*"screen" + 0.007*"kindle" + 0.006*"read" + 0.006*"good" + 0.005*"device" + 0.005*"would" + 0.005*"great" + 0.005*"cable"')
(6, '0.031*"nook" + 0.011*"read" + 0.0

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [31]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Coherence score in topic modeling to measure how interpretable the topics are to humans.
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(corpus, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=data_words, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))
    


Coherence score with 2 clusters: 0.3414233754473808
Coherence score with 3 clusters: 0.49122267044748624
Coherence score with 4 clusters: 0.46443645471335915
Coherence score with 5 clusters: 0.35431868286442
Coherence score with 6 clusters: 0.41092451538696
Coherence score with 7 clusters: 0.36779228068014247
Coherence score with 8 clusters: 0.34807734691444825
Coherence score with 9 clusters: 0.3588142453677833
Coherence score with 10 clusters: 0.3320485338768269


In [34]:
# perform SVD on the bag of words with the LsiModel to extract 3 topics
lsi = LsiModel(corpus, num_topics=3, id2word=dictionary)

In [35]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.620*"nook" + 0.253*"books" + 0.211*"kindle" + 0.165*"read" + 0.165*"screen" + 0.157*"like" + 0.134*"book" + 0.127*"one" + 0.108*"device" + 0.103*"tablet".
Words in 1: -0.595*"nook" + 0.227*"device" + 0.208*"android" + -0.162*"kindle" + 0.129*"one" + 0.124*"screen" + 0.120*"would" + 0.111*"nice" + 0.106*"tablet" + 0.104*"home".
Words in 2: 0.395*"tablet" + -0.339*"books" + 0.273*"android" + 0.264*"nook" + 0.174*"apps" + -0.174*"read" + -0.168*"kindle" + -0.152*"book" + 0.146*"hd" + 0.137*"music".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [59]:
pip install --upgrade lda2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('amazon_reviews.csv')
df = df.dropna()
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09
1,1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09
2,2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09
3,3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1.290557e+09
4,4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1.317254e+09


In [72]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df['reviewText'].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,clean_text
0,0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09,got gps husband otr road trucker impressed shi...
1,1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09,'m professional otr truck driver bought tnd tr...
2,2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09,well say 've unit truck four days prior garmin...
3,3,0528881469,"[9, 10]",2.0,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1.290557e+09,going write long review even thought unit dese...
4,4,0528881469,"[0, 0]",1.0,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1.317254e+09,'ve mine year 's got tries route non truck rou...


In [77]:
!pip install --upgrade joblib
!pip install bertopic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
from bertopic import BERTopic

In [75]:
tweets = df.clean_text.to_list()

In [80]:
topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

In [89]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,14,-1_ipod_hd_video_thailand
1,0,456,0_nook_books_nt_kindle
2,1,222,1_tv_mount_wall_mounting
3,2,120,2_cable_hdmi_canon_works
4,3,44,3_dvd_drive_cd_netbook
5,4,43,4_mic_ipod_microphone_skype
6,5,42,5_usb_adapter_cable_plug
7,6,26,6_card_galaxy_samsung_phone
8,7,25,7_batteries_charge_charger_battery


In [90]:
topic_model.visualize_topics()

In [91]:
topic_model.get_topic(0)


[('nook', 0.05002798009031621),
 ('books', 0.030039940223332465),
 ('nt', 0.02964677632928862),
 ('kindle', 0.02834318618680163),
 ('screen', 0.023701034949746583),
 ('read', 0.022691983292404013),
 ('like', 0.020831098572975815),
 ('book', 0.019421604987995205),
 ('tablet', 0.01892387671910334),
 ('one', 0.01844107057818852)]

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

For topic modeling, BERTopic seems to be the superior approach, especially for bigger datasets. It may capture the semantic meaning of words and produces subjects that are more coherent and interpretable without requiring extensive hyperparameter optimization. LSA may be a superior solution, especially for smaller datasets, if computing economy is an issue. LDA has been utilized extensively in research and continues to be a good option for topic modeling.
The optimal topic modeling algorithm to use will rely on the particular requirements of the application. LDA should be used if interpretability is the key issue. In cases where domain-specific information is crucial, BERTopic and Lda2Vec might be preferable. LDA and BERTopic are the best options if coherence score is a primary priority.